# mdx VMのデプロイとVCP既存サーバ (SSH) モード セットアップ

この Notebook では、mdx VM のデプロイと、その mdx VM をVCP既存サーバ(SSH) モードで利用できるようにするまでのセットアップを行う。

## 前提条件

1. VCコントローラが動作していること
    * このNotebookを実行しているJupyterNotebookサーバと同一ホスト上で稼働するPortable版VCコントローラの利用を想定する。
2. vcp_config.yml においてVCコントローラの設定が完了していること

## 準備

1. この Notebook を実行する Jupyter Notebook サーバに `expect` パッケージをインストール
2. mdx のローカル認証により REST API を利用するためのパラメータ設定
3. mdx VM にSSHログインするためのキーペア作成
4. mdx の REST API エンドポイントに接続できることの確認

### expect のインストール確認

In [ ]:
!expect -v

In [ ]:
# 未インストールの場合
!sudo apt update && apt install -y expect

### mdx REST API パラメータ設定

- mdx REST API 認証トークンは、mdxユーザポータルの「トークン発行」により取得する。
  * 「トークン発行」は、ユーザポータル画面右上にあるユーザ名をクリックすると表示される。

In [ ]:
# mdx REST API 認証トークン設定
from getpass import getpass
mdx_token = getpass("mdx API token")

In [ ]:
# mdx VMに設定する初期パスワード
mdx_user_password = "Pyn5jzxM"

# mdx VM名　(project内で一意になるようにする)
vm_name = "cwh-node01"

# Notebookからmdx VMにsshログインするssh鍵ペア
ssh_private_key_path = "~/.ssh/id_rsa"
ssh_public_key_path = "~/.ssh/id_rsa.pub"

# mdx VMのログインユーザ名
# mdxカタログ情報の "login_username" から取得するか、またはユーザポータルの「仮想マシンテンプレート」で確認できる。
ssh_user_name = "mdxuser"

### mdx VM への SSH ログイン用キーペア作成

In [ ]:
!test -f ~/.ssh/id_rsa || ssh-keygen -t rsa -f ~/.ssh/id_rsa -N ""
!ls -l ~/.ssh

### mdx REST API エンドポイント接続確認

In [ ]:
# デフォルトのresolverがIPv6のアドレスを返すが、接続できないときに以下のコードを実行する
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

## mdx VM 作成

1. VCP SDK mdx用プラグインモジュールの読み込み
2. mdx VM作成に必要なパラメータ確認
3. mdx VMデプロイ

### VCP SDK mdx用プラグインモジュールの読み込み

- 以下の .py ファイルが必要。
  * `/home/jovyan/vcpsdk/vcpsdk/plugins/mdx_{ext,lib}.py`
  * `/home/jovyan/vcpsdk/common/logsetting.py`

In [ ]:
from common import logsetting
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### mdx VM作成に必要なパラメータ確認

- プロジェクトID
- ネットワークセグメントID

In [ ]:
projects = mdx.get_assigned_projects()
projects

In [ ]:
# mdxのプロジェクトのIDを取得 (先頭ID)
project_id = projects[0]["projects"][0]["uuid"]
project_id

In [ ]:
# mdxの操作を行うプロジェクトを設定
mdx.set_current_project_id(project_id)

In [ ]:
# mdxのプロジェクトに割り当てられたネットワークのうち、先頭のネットワークセグメントのidを取得
segment_id = mdx.get_segments()[0]["uuid"]
segment_id

In [ ]:
# すでに稼働中のmdx VM一覧を取得し表示する
import json
lst = mdx.get_vm_list()
print(json.dumps(lst, indent=2))

### mdx VMデプロイ

- VM起動後、SSH接続可能な IPv4 アドレスの割り当てに5分程度を要する。

In [ ]:
# 仮想マシンテンプレート (推奨版 東京大学制作、20220412-2043版)
DEFAULT_CATALOG = "16a41081-a1cf-428e-90d0-a147b3aa6fc2"
DEFAULT_TEMPLATE_NAME = "UT-20220412-2043-ubuntu-2004-server"

# ssh公開鍵
import os
with open(os.path.expanduser(ssh_public_key_path)) as f:
    ssh_shared_key = f.read()

# VMスペック指定
mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=16,
    pack_type="cpu",
    disk_size=80,
    gpu="0",

    network_adapters=[
        dict(
            adapter_number=1,
            segment=segment_id
        )
    ],
    shared_key=ssh_shared_key,

    # VCPのmdx拡張で以下は固定値とする
    # os_type="Linux",
    # power_on=True,
    # project=project_id,
    storage_network="portgroup",
)
info = mdx.deploy_vm(vm_name, mdx_spec)

# mdx VMのdeployが完了
print(json.dumps(info, indent=2))

In [ ]:
# 確保したmdx VMのプライベートIPv4アドレスを取得
host_ip_address = info["service_networks"][0]["ipv4_address"][0]
host_ip_address

## VCP既存サーバ (SSH) モード セットアップ

1. mdx VMへの疎通確認
2. VM初期パスワード設定
3. 既存サーバ(SSH) モード セットアップスクリプトをmdx VMへのコピー
4. 既存サーバ(SSH) モード セットアップ実行
5. Docker のインストール確認

### mdx VMへの疎通確認

In [ ]:
!ping -c 3 {host_ip_address}

### VM初期パスワード設定

In [ ]:
!./scripts/init_mdx_passwd.exp {ssh_user_name} {host_ip_address} {ssh_private_key_path} {mdx_user_password}

### 既存サーバ(SSH) モード セットアップスクリプトをmdx VMへのコピー

In [ ]:
# notebookからmdx VMへのsshコマンドオプション
ssh_opts = f" -i {ssh_private_key_path} -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null"

In [ ]:
!scp -i tmp/id_rsa -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null scripts/init_mdx_node.sh {ssh_user_name}@{host_ip_address}:

### 既存サーバ(SSH) モード セットアップ実行

- docker-ce インストール
- sshd_config 設定変更: Port 20022 

In [ ]:
!ssh {ssh_opts} {ssh_user_name}@{host_ip_address} ./init_mdx_node.sh

### Docker のインストール確認

In [ ]:
!ssh {ssh_opts} -p 20022 {ssh_user_name}@{host_ip_address} sudo docker ps

## VCP SDKによるVCノード起動確認

1. VCP SDK 初期化
2. Portable VCコントローラ公開鍵設定
3. VCノード起動パラメータ設定
4. VCノード起動
5. Applicationコンテナの起動確認

### VCP SDK 初期化

In [ ]:
# VCP REST API アクセストークンの入力
from getpass import getpass
vcc_access_token = getpass()

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
sdk = VcpSDK(vcc_access_token)

# VCP SDK バージョン確認
sdk.version()

### VCコントローラ公開鍵設定

VCコントローラがmdx VMを既存サーバ(SSH) モードで制御するための SSH 公開鍵を mdx VM に設定する。

In [ ]:
# VCコントローラが既存サーバを操作するときに使うssh公開鍵を取得する
vcc_ssh_pubkey = sdk.get_publickey()

In [ ]:
!echo "{vcc_ssh_pubkey}" | (ssh {ssh_opts} -p20022 {ssh_user_name}@{host_ip_address} "cat >> ~/.ssh/authorized_keys")

### VCノード起動パラメータ設定

- Unit Group 作成
- Baseコンテナへのsshログイン用鍵ペア作成
- Unit の spec 指定

In [ ]:
my_ugroup_name = "mdx"
ugroup = sdk.create_ugroup(my_ugroup_name)

In [ ]:
print(ugroup)

In [ ]:
# baseコンテナへのsshログインの鍵を作成
if not os.path.exists("base_tmp/"):
    !mkdir base_tmp
    !ssh-keygen -t rsa -b 2048 -N "" -f base_tmp/id_rsa_base

In [ ]:
spec = sdk.get_spec("onpremises", "default")
spec.ip_addresses = [host_ip_address]
spec.user_name = ssh_user_name
spec.set_ssh_pubkey("base_tmp/id_rsa_base.pub")

### VCノード起動

In [ ]:
unit_name = "mdx_server"
unit = ugroup.create_unit(unit_name, spec, wait_for=True, verbose=0)

In [ ]:
ugroup.df_nodes()

### Applicationコンテナの起動確認

`hello-world` コンテナを実行する。

In [ ]:
!ssh -i base_tmp/id_rsa_base -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null root@{host_ip_address} /usr/local/bin/docker run hello-world

## VCノード削除

ここでは VC ノードとしての動作確認を行った Unit Group の cleanup を実行するが、
VCP既存サーバ(SSH) モードをセットアップ済みの mdx VM は VC ノードとして再利用可能である。

In [ ]:
ugroup.cleanup()

In [ ]:
sdk.df_ugroups()